In [2]:
import datetime
import random
import time
import baostock as bs
import pandas as pd
from pandas.tseries.offsets import BDay
from tqdm import tqdm

In [3]:
def near_year_lowest_price(code, threshold=0.1):
    start = (datetime.date.today() - datetime.timedelta(weeks=52)).isoformat()
    rs = bs.query_history_k_data_plus(code, "close", start, "", adjustflag="2")
    stock_price = rs.get_data()
    if 'close' not in stock_price:
        return False
    lowest = stock_price['close'][:].astype('float64').min()
    latest = stock_price['close'].astype('float64').iloc[-1]
    return lowest * (1 + threshold) > latest


def get_stock_list():
    today = datetime.datetime.today()
    trade_day = (today - BDay(1)).strftime('%Y-%m-%d')
    print("交易日", trade_day)
    stocks = []
    l = bs.login()
    rs = bs.query_all_stock(trade_day)
    while (rs.error_code == '0') & rs.next():
        stocks.append(rs.get_row_data())            
    l = bs.logout()
    
    result = pd.DataFrame(stocks, columns=rs.fields)
    result = result[['code', 'code_name']]
    result = result[result['code_name'].str.contains('指数') == False]
    result = result[result['code_name'].str.contains('债') == False]
    return result


In [4]:
stock_list = get_stock_list()
print("股票数", len(stock_list))

near_low_list = []
code_name_list = []
black_list = pd.read_csv('blacklist.csv')
l = bs.login()
for i in tqdm(range(len(stock_list))):
    code = stock_list.iloc[i]['code']
    code_name = stock_list.iloc[i]['code_name']
    if code in black_list['code'].values:
        continue
        
    if near_year_lowest_price(code):
        near_low_list.append(code)
        code_name_list.append(code_name)
        
    if i % 57 == 48:
        time.sleep(random.random() * 2)
l = bs.logout()
print("低价股票数", len(near_low_list))

d = {'code': near_low_list, 'code_name': code_name_list}
df = pd.DataFrame(data=d)
df.to_csv('low_price_stock_list.csv', index=False)

交易日 2024-08-16
login success!
logout success!
股票数 5186
login success!


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5186/5186 [05:07<00:00, 16.87it/s]

logout success!
低价股票数 1881
